In [16]:
import os, cv2, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils

In [17]:
def process_image_test(directory):
    images = []
    labels = []
    for root, _, files in os.walk(directory):
            counter = 0
            for file_name in files:
                counter +=1
                if counter <=300: #read only first 300 images for less time consumption
                    file_path = os.path.join(root, file_name)
                    images.append(file_path)
                    labels.append(file_path.split('/')[2])
                else:
                    break;
    return images,labels

In [18]:
train_image_geo,train_image_label = process_image_test('Data/Train')

In [19]:
test_image_geo,test_image_label = process_image_test('Data/Test')

In [21]:
ROWS = 64
COLS = 64
CHANNELS = 3
def resize_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR) 
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

In [24]:
def convert_image_to_array(images):
    count = len(images)
    data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype=np.uint8)

    for i, image_file in enumerate(images):
        image = resize_image(image_file)
        data[i] = image.T
        if i%500 == 0: print('Processed {} of {}'.format(i, count))
    
    return data

In [25]:
train_image = convert_image_to_array(train_image_geo)
test_image = convert_image_to_array(test_image_geo)

Processed 0 of 1500
Processed 500 of 1500
Processed 1000 of 1500
Processed 0 of 1329
Processed 500 of 1329
Processed 1000 of 1329


In [27]:
print("Train shape: {}".format(train_image.shape))
print("Test shape: {}".format(test_image.shape))

Train shape: (1500, 3, 64, 64)
Test shape: (1329, 3, 64, 64)


In [28]:
train_label_df = pd.DataFrame({'Label':train_image_label},columns=['Label'])
train_label_df['Label_numeric'] = train_label_df.Label.map({"Circle":0,"Hexagon":2,"Pentagon":3,"Rectangle":1,"Triangle":5})

train_label = [str(i) for i in train_label_df['Label_numeric']]

In [29]:
test_label_df = pd.DataFrame({'Label':test_image_label},columns=['Label'])
test_label_df['Label_numeric'] = test_label_df.Label.map({"Circle":0,"Hexagon":2,"Pentagon":3,"Rectangle":1,"Triangle":5})

test_label = [str(i) for i in test_label_df['Label_numeric']]

In [32]:
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'


def catdog():
    
    model = Sequential()

    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(3, ROWS, COLS), activation='relu'))
    model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
    return model


model = catdog()

In [ ]:
def get_predictions_score():
    
    model.fit(train_image, train_label, validation_data=(test_image, test_label), 
              nb_epoch=10, batch_size=200, verbose=2)
#     model.fit(train_image, train_label, batch_size=batch_size, nb_epoch=nb_epoch,
#               validation_split=0.25, verbose=0, shuffle=True, callbacks=[history, early_stopping])
    

    #predictions = model.predict(test_image, verbose=0)
    scores = model.evaluate(test_image, test_label, verbose=0)
    return scores

predictions = get_predictions_score()

In [ ]:
predictions